In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from pathlib import Path

from tensorflow import Tensor
from tqdm.notebook import tqdm
from typing import Callable, List, Tuple, Union, Optional, Dict, Any, Sequence, Iterable, TypeVar
from DataPipe import DataPipeFactory
from util_function import inform_pooling, Get_Gradient

/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/p

In [2]:
# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [3]:
#####
#####6824149711

In [4]:
ds = DataPipeFactory('../DataFolder/Tensorflow_DataRecord/Student_Answer_Record.tfrecord',
                     '../DataFolder/Siri_Related/Siri_Reference_Sample',
                     '../DataFolder/Siri_Related/Siri_Dense_Index', cache='../DataFolder/cache/datapipe/cached')
# dsp = ds.get_batch_data(10)
# it = iter(dsp)
ds.get_raw_data()
A = tf.Variable(-1.)

In [5]:
#ds.pre_save()

In [6]:
iter(ds.get_batch_data(10, interleave=True))

In [7]:
for i,d in tqdm(enumerate(ds.get_batch_data(10, interleave=False, deterministic=False))):
    if i % 50 == 0:
        print(i)

0it [00:00, ?it/s]

2023-06-09 13:51:13.686815: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-09 13:51:14.340295: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:362 : INVALID_ARGUMENT: Detected unsupported operations when trying to compile graph __inference___single_mapping_754[] on XLA_CPU_JIT: _Arg (No registered '_Arg' OpKernel for XLA_CPU_JIT devices compatible with node {{node main_7}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_STRING, _output_shapes=[[]], _user_specified_name="main", index=2){{node main_7}}
2023-06-09 13:51:14.696891: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:362 : INVALID_ARGUMENT: Detected unsupported operations when trying to compile graph __inference___single_mapping_754[] on XLA_CPU_JIT: _Arg (No registered '_Arg' OpKernel for XLA_CPU_JIT devices compatible with node {{node main_7}}
	 (OpKernel was found, but attributes

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_7_device_/job:localhost/replica:0/task:0/device:CPU:0}} Detected unsupported operations when trying to compile graph __inference___single_mapping_754[] on XLA_CPU_JIT: _Arg (No registered '_Arg' OpKernel for XLA_CPU_JIT devices compatible with node {{node main_7}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_STRING, _output_shapes=[[]], _user_specified_name="main", index=2){{node main_7}}
	 [[StatefulPartitionedCall]] [Op:IteratorGetNext]

In [ ]:
for i,d in tqdm(enumerate(ds.get_batch_data(10))):
    batch = 10
    value = tf.RaggedTensor.from_tensor(d['stu_mfcc'], padding=-1*tf.ones(80,))
    start = tf.RaggedTensor.from_tensor(d['valid_stu_start'], padding=-1.)
    duration = tf.RaggedTensor.from_tensor(d['valid_stu_duration'], padding=-1.)
    ratio = 125
    #Get_Gradient(value, start, duration, ratio)
    # print(i)
    # with tf.GradientTape() as t:
    #     v2 = value * A
    #     Final_Tensor = inform_pooling(batch, v2, start, duration, ratio)
    #     J = tf.reduce_mean(Final_Tensor)
    # print('Start Gradient')
    # print(f'\tLoss Value: {J}, \n\tOutput Shape {tf.shape(Final_Tensor)}')
    # G = t.gradient(J,A)
    # print(i,G)
    # print('End Gradient\n\n')
    # del t
    # print(G)
    if i ==30 :break
    pass

In [ ]:
d.keys()

In [ ]:
class InformPooling(tf.keras.layers.Layer):
    def __init__(self, num_maps, ratios_list, **kwargs):
        super().__init__(**kwargs)
        self.num_maps = num_maps
        self.ratios_list = ratios_list

    @staticmethod
    @tf.function
    def inform_pooling(value, start, duration, ratio, eps=0.001):
        batch = tf.shape(value)[0]
        end = start + duration
        start = tf.math.floor(start * ratio)
        end = tf.math.ceil((end + eps) * ratio)
        period = tf.cast(tf.stack([start, end], axis=-1), tf.int32)
        tf.debugging.assert_less(period[..., 0], period[..., 1])
        ret_b = tf.TensorArray(tf.float32, batch, infer_shape=False)
        ret_count = tf.TensorArray(tf.int64, batch)
        for batch_index in tf.range(batch):
            value_l = value[batch_index]
            val_ind_max = tf.shape(value_l)[0]
            period_l = period[batch_index]
            period_l_p = tf.math.minimum(period_l, val_ind_max - 1)
            ret_count = ret_count.write(batch_index, tf.cast(tf.shape(period_l)[0], tf.int64))
            indexes = tf.ragged.range(period_l_p[..., 0], period_l_p[..., 1])
            value_indices = tf.gather(value_l, indexes)
            pooled = tf.reduce_mean(value_indices, axis=1)
            ret_b = ret_b.write(batch_index, pooled)
        row_length = ret_count.stack()
        ret = ret_b.concat()
        return ret, row_length

    @tf.function
    def call(self, value_list, start, duration):
        # Iterate over both value_list and ratio_list
        pooled_value = [self.inform_pooling(value, start, duration, ratio) for (value, ratio) in
                        zip(value_list, self.ratios_list)]
        ret = tf.concat([val for val, _ in pooled_value], axis=-1)
        # Stupid way to shrink dynamic shape to static shape
        return tf.RaggedTensor.from_row_lengths(ret, pooled_value[0][1])

In [ ]:
apl = InformPooling(3,[125 / 2**i for i in range(3)])

In [ ]:
def getinfo(d,str_value, str_start, str_duration, str_words):
    batch = 10
    value = tf.RaggedTensor.from_tensor(d[str_value], padding=-1*tf.ones(80,))
    start = tf.RaggedTensor.from_tensor(d[str_start], padding=-1.)
    duration = tf.RaggedTensor.from_tensor(d[str_duration], padding=-1.)
    ratio = 125
    words = tf.RaggedTensor.from_tensor(d[str_words], padding=-1)
    end = start + duration
    period = tf.cast(tf.stack([start, end], axis=-1) * ratio, tf.int64)
    return apl([value]*3, start, duration), words

In [ ]:
v1, wo1 = getinfo(d,'stu_mfcc','valid_stu_start','valid_stu_duration', 'valid_ref_word')

In [ ]:
d.keys()

In [ ]:
v2, wo2 = getinfo(d, 'ref_mfcc', 'valid_ref_start', 'valid_ref_duration', 'valid_ref_word')

In [ ]:
tf.shape(v1)

In [ ]:
tf.shape(v2)

In [ ]:
@tf.function
def compute_similarity(value_a,value_b, ref_a, ref_b, margin=0.25, eps=0.001):
    #if ref_a equal ref_b then we consider it should be similar else it should be different,
    #margin prevent it been push to far away
    # compute the norm for ragged tensor
    norm_of_a = tf.sqrt(tf.reduce_sum(tf.square(value_a),axis=-1, keepdims=True))
    norm_of_b = tf.sqrt(tf.reduce_sum(tf.square(value_b),axis=-1, keepdims=True))
    norm_a = value_a / (norm_of_a + eps)
    norm_b = value_b / (norm_of_b + eps)
    # compute cosine similarity for each sample in batch
    # get batch size
    batch_size = tf.shape(norm_a)[0]
    loss_array = tf.TensorArray(tf.float32, batch_size, infer_shape=False)
    for idx in tf.range(batch_size):
        va = norm_a[idx]
        vb = norm_b[idx]
        ra = ref_a[idx]
        rb = ref_b[idx]
        similarity_matrix = tf.matmul(va,vb, transpose_b=True)
        # compute the mask for the positive samples
        mask = tf.cast(tf.equal(ra,rb),tf.float32)
        # compute the mask for the negative samples
        mask_neg = tf.cast(tf.not_equal(ra,rb),tf.float32)
        # compute the number of positive and negative samples
        num_pos = tf.cast(tf.reduce_sum(mask), tf.float32)
        num_neg = tf.cast(tf.reduce_sum(mask_neg), tf.float32)
        # compute the average similarity for the positive samples
        # avoid 0
        num_pos = tf.maximum(num_pos, 1.)
        num_neg = tf.maximum(num_neg, 1.)
        avg_sim_pos = tf.reduce_sum(tf.multiply(similarity_matrix,mask)) / num_pos
        # compute the average similarity for the negative samples
        avg_sim_neg = tf.reduce_sum(tf.multiply(similarity_matrix,mask_neg)) / num_neg
        # compute the max similarity for the positive samples
        max_sim_pos = tf.reduce_max(tf.multiply(similarity_matrix,mask))
        # compute the min similarity for the negative samples
        min_sim_neg = tf.reduce_min(tf.multiply(similarity_matrix,mask_neg))
        # compute the average loss with margin
        loss_avg = tf.maximum(0., margin + avg_sim_pos - avg_sim_neg)
        # compute min_max loss with margin
        loss_min_max = tf.maximum(0., margin + max_sim_pos - min_sim_neg)
        # total loss
        loss = loss_avg + loss_min_max
        loss_array = loss_array.write(idx,loss)
    total_loss = tf.reduce_sum(loss_array.stack())
    return total_loss

In [ ]:
compute_similarity(v1,v2,wo1,wo2)

In [ ]:
d['valid_ref_word']

In [ ]:
c = tf.matmul(v1,v2, transpose_b=True)

In [ ]:
c.nested_row_lengths()

In [ ]:
rt = tf.RaggedTensor.from_row_splits(
    values=tf.RaggedTensor.from_row_splits(
        values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        row_splits=[0, 3, 3, 5, 9, 10]),
    row_splits=[0, 1, 1, 5])

In [ ]:
rt.nested_row_lengths()

In [ ]:
ap = inform_pooling(value, start, duration, ratio)

In [ ]:
tf.shape(ap)

In [ ]:
tf.shape(tf.concat([ap,ap],axis=-1))

In [ ]:
down_sample_layer = tf.keras.layers.Conv1D(120,kernel_size=3,strides=2,padding="same")

In [ ]:
up_sample_layer = tf.keras.layers.Conv1DTranspose(120,kernel_size=4,strides=2,padding="same")

In [ ]:
i = d['stu_mfcc']

In [ ]:
s = down_sample_layer(i)

In [ ]:
u = up_sample_layer(s)

In [ ]:
u

In [ ]:
i

In [ ]:
u[:,:tf.shape(i)[2],:]

In [ ]:
tf.ensure_shape(i)

In [ ]:
@tf.function
def mt(i):
    shape = tf.shape(i)[1]
    s = down_sample_layer(i)
    u = up_sample_layer(s)
    cut = u[:,:shape]
    return tf.concat([i,cut],axis=-1)


In [ ]:
mt(i)

In [ ]:
shape = tf.shape(i)[:-1]
cut = tf.slice(u, tf.zeros_like(shape), shape)

In [ ]:
shape

In [ ]:
rt = tf.RaggedTensor.from_uniform_row_length(
    values=tf.RaggedTensor.from_row_splits(
        values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        row_splits=[0, 3, 5, 9, 10]),
    uniform_row_length=2)

In [ ]:
rt.nested_row_splits

In [ ]:
rt.row_splits

In [ ]:
rt.values

In [ ]:
rt

In [ ]:
dst, dse = ds.k_fold(10,1,32)

In [ ]:
exampl = next(iter(dst))

In [ ]:
exampl.keys()